In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
from reproject import reproject_interp
from astropy import stats
import os 
from astropy.table import QTable, join, vstack, hstack, Column
from tqdm import tqdm

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [3]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                if '628e' in file: 
                    continue
                else: 
                    print(os.path.join(root, file))
                    file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_offset = get_files('table_fits_filteranchor.fits', root_dir)
files_slope = get_files('table_fits_haanchor.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/table_fits_filteranchor.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1087/hst_contsub/table_fits_filteranchor.fits


In [4]:
gals  = []
output_stats = []

for file_offset, file_slope in tqdm(zip(files_offset, files_slope)): 

    # Loading files...
    table_offset = QTable.read(file_offset)
    table_slope = QTable.read(file_slope)

    gal1 = file_offset.split('/')[-3]
    gal2 = file_offset.split('/')[-3]

    if gal1 != gal2: 
        print('ERROR - galaxies do not match!')
        break

    gals += [gal1]

    intercepts = np.float32(table_offset['intercept_lowres'].value)
    slope = np.float32(table_slope['slope_bins'].value[1]) # for anchored image only
    stats = np.array([intercepts[0], intercepts[1], intercepts[2], slope])

    output_stats += [stats]

gals = np.array(gals, dtype='<U8')
output_stats = np.array(output_stats)

19it [00:00, 123.67it/s]


In [5]:
# Get into a astropy table the output of the stats of the maps to .tex

gal_names = gals.copy()
for i, gal in enumerate(gal_names):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    # if '628C' in gal:
    #     gal_split = gal.split('628C')
    #     gal = gal_split[0] + '0628'
    gal_names[i] = gal

names=('F555W', 'F65XN', 'F814W', 'N[II] corr')

table1 = Column(gal_names, name='Galaxy')
table2 = QTable(output_stats, names=names)

table = hstack([table1, table2])
table.sort('Galaxy')
table

Galaxy,F555W,F65XN,F814W,N[II] corr
str8,float32,float32,float32,float32
IC~5332,-11.076195,66.90957,-7.534602,1.0806233
NGC~1087,-8.619963,36.11675,-5.8762546,1.1228473
NGC~1300,-3.2584872,-3.7176487,-1.9037491,1.2121018
NGC~1365,-30.679022,22.67622,-27.017344,1.2230378
NGC~1385,-8.537724,36.565125,-5.756293,1.1800598
NGC~1433,-24.415028,-24.494146,-20.859722,1.3324771
NGC~1512,-6.6066236,51.97614,-5.5493546,1.0277058
NGC~1566,-41.089554,72.81185,-22.079185,1.0052222
NGC~1672,-8.582992,-14.249862,-6.5455933,1.3325003


In [6]:
formats={
        'Galaxy': '%s',
        'F555W': '%.2f',
        'F65XN': '%.2f',
        'F814W': '%.2f',
        'N[II] corr': '%.2f'
        }

table.write('./tabs/galfit_props.tex', formats=formats, overwrite=True)

In [7]:
np.nanmean(table['N[II] corr'])

1.0923421

In [14]:
gals  = []
output_stats = []

for file_offset, file_slope in tqdm(zip(files_offset, files_slope)): 

    # Loading files...
    table_offset = QTable.read(file_offset)
    table_slope = QTable.read(file_slope)

    gal1 = file_offset.split('/')[-3]
    gal2 = file_offset.split('/')[-3]

    if gal1 != gal2: 
        print('ERROR - galaxies do not match!')
        break

    gals += [gal1]

    intercepts = np.float32(table_offset['interceptfixed_bins'].value)
    slope = np.float32(table_slope['slopefixed_bins'].value[1]) # for anchored image only
    stats = np.array([intercepts[0], intercepts[1], intercepts[2], slope])

    output_stats += [stats]

gals = np.array(gals, dtype='<U8')
output_stats = np.array(output_stats)

19it [00:00, 115.29it/s]


In [17]:
# Get into a astropy table the output of the stats of the maps to .tex

gal_names = gals.copy()
for i, gal in enumerate(gal_names):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    # if '628C' in gal:
    #     gal_split = gal.split('628C')
    #     gal = gal_split[0] + '0628'
    gal_names[i] = gal

names=('F555W fix', 'F65XN fix', 'F814W fix', 'N[II] corr fix')

table1 = Column(gal_names, name='Galaxy')
table2 = QTable(output_stats, names=names)

table = hstack([table1, table2])
table.sort('Galaxy')
table

Galaxy,F555W fix,F65XN fix,F814W fix,N[II] corr fix
str8,float32,float32,float32,float32
IC~5332,-12.997702,72.76199,-8.577524,1.0582305
NGC~1087,-9.609122,34.63646,-6.4033437,1.1164248
NGC~1300,-2.5876772,-3.7403789,-2.2415109,1.2584573
NGC~1365,-33.437202,19.189957,-28.230797,1.236707
NGC~1385,-9.493533,35.258614,-6.8105783,1.1935498
NGC~1433,-24.756039,-26.027035,-21.65933,1.4288096
NGC~1512,-7.497829,56.40855,-5.415182,1.014432
NGC~1566,-40.940517,65.640564,-22.739353,1.007966
NGC~1672,-9.160834,-14.386707,-6.757429,1.3377496


In [18]:
formats={
        'Galaxy': '%s',
        'F555W fix': '%.2f',
        'F65XN fix': '%.2f',
        'F814W fix': '%.2f',
        'N[II] corr fix': '%.2f'
        }

table.write('./tabs/galfitfixed_props.tex', formats=formats, overwrite=True)